In [81]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

URL = "https://www.npci.org.in/what-we-do/upi/upi-ecosystem-statistics"
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
def get_soup(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

In [82]:
table_headings = [' Sr. No. ', ' UPI Remitter Banks  ', ' Total Volume(In Mn) ', ' Approved % ', ' BD % ', ' TD% ', ' Debit Reversal Success % '][:6]
def table_to_df(table):
    table_name = table['id'][4:]
    table_name = table_name[-4:] + table_name[:-4]
    all_rows = table.find_all('tr')
    processed_data = []
    
    # Only using top 30 banks
    if all_rows[1].find_all('td'): all_rows = all_rows[1:31]
    else: all_rows = all_rows[2:32]

    for row in all_rows:
        data = [x.text for x in row.find_all('td')]
        data = [y.replace(',', '') for y in [x.replace('%', '') for x in data]]
        processed_data.append(data[:6])
    df = pd.DataFrame(processed_data, columns = table_headings)
    return table_name, df
        

In [83]:
all_tables = soup.find_all('div', class_='hideDD')
yearwise_tables = {}
for table in all_tables[1:]:
    name, df = table_to_df(table)
    # df.to_csv(f'Data/{name}.csv', index=False)
    yearwise_tables[name] = df
